In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


monday = pd.read_csv('/Users/krystanafoh/repos/KMP-simulation/data/monday.csv', delimiter =';')
tuesday = pd.read_csv('/Users/krystanafoh/repos/KMP-simulation/data/tuesday.csv', delimiter =';' )
wednesday = pd.read_csv('/Users/krystanafoh/repos/KMP-simulation/data/wednesday.csv', delimiter =';')
thursday = pd.read_csv('/Users/krystanafoh/repos/KMP-simulation/data/thursday.csv', delimiter =';')
friday = pd.read_csv('/Users/krystanafoh/repos/KMP-simulation/data/friday.csv', delimiter =';')
df = pd.concat([monday, tuesday, wednesday, thursday, friday], axis = 0)

df['timestamp'] = pd.to_datetime(df['timestamp'])

conditions = [
    (df['location'] == "fruit"),
    (df['location'] == "spices"),
    (df['location'] == "dairy"),
    (df['location'] == "drinks"),
    ]

values = [4, 3, 5, 6]

#df['revenue_per_minute'] = np.select(conditions, values)

#df["unq_id"] = 'Cust_' + df["timestamp"].astype(str) + '_no_' + df["customer_no"].astype(str)   !!!! TOO UNIQUE TO GROUPBY

df = df[['timestamp', 'customer_no', 'location' ]]

#######################
# trans prop
########################

df_tp = df.sort_values(['customer_no', 'timestamp'])
df_tp.set_index('timestamp', inplace=True)


In [3]:
df_tp

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:05:00,1,checkout
2019-09-03 07:02:00,1,fruit
2019-09-03 07:05:00,1,drinks
2019-09-03 07:12:00,1,checkout
...,...,...
2019-09-05 21:48:00,1533,dairy
2019-09-05 21:48:00,1534,fruit
2019-09-05 21:49:00,1534,checkout


In [4]:
df_tp['day'] = df_tp.index.day

In [5]:
df_tp['unq_id'] = 'Cust' + df_tp["day"].astype(str) + df_tp["customer_no"].astype(str)

In [6]:
df_tp

,customer_no,location,day,unq_id
timestamp,,,,
2019-09-02 07:03:00,1,dairy,2,Cust21
2019-09-02 07:05:00,1,checkout,2,Cust21
2019-09-03 07:02:00,1,fruit,3,Cust31
2019-09-03 07:05:00,1,drinks,3,Cust31
2019-09-03 07:12:00,1,checkout,3,Cust31
...,...,...,...,...
2019-09-05 21:48:00,1533,dairy,5,Cust51533
2019-09-05 21:48:00,1534,fruit,5,Cust51534
2019-09-05 21:49:00,1534,checkout,5,Cust51534


In [7]:
df_tp = df_tp.groupby('customer_no').resample('1min').fillna('ffill')
df_tp['before'] = df_tp['location'].shift(1)

In [8]:
trans_prob = pd.crosstab(df_tp['location'], df_tp['before'], normalize=0)
#trans_prob.to_csv('data/trans_matrix_prob.csv', index=True)

In [9]:
trans_prob

before,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,0.999121,0.000216,0.000248,0.000303,0.000111
dairy,0.061823,0.864568,0.003069,0.035356,0.035183
drinks,0.116492,0.107547,0.598705,0.071664,0.105593
fruit,0.104721,0.033201,0.031932,0.808821,0.021325
spices,0.100893,0.068914,0.063509,0.048495,0.718189


In [11]:
# write first location for each customer to list
first_locations = df_tp.groupby('unq_id').location.first().to_list()

In [12]:
# create dataframe out of list 
df_firsts = pd.DataFrame(first_locations)

In [13]:
df_firsts

,0
0,dairy
1,fruit
2,drinks
3,dairy
4,fruit
...,...
7440,spices
7441,spices
7442,fruit
7443,drinks


In [32]:
check =df_firsts[0].value_counts(normalize=True).index.to_list()

In [33]:
check

['fruit', 'dairy', 'spices', 'drinks']

In [18]:
# get value counts in percentage == probabilty for next after entrance to dictionary
entrance_p ={}

for idx, loc in enumerate (df_firsts[0].value_counts(normalize=True).index.to_list()): 
    
    entrance_p[loc] = (df_firsts[0].value_counts(normalize=True)[idx])

In [19]:
entrance_p

{'fruit': 0.3774345198119543,
 'dairy': 0.2875755540631296,
 'spices': 0.18146406984553393,
 'drinks': 0.15352585627938214}

In [20]:
entrance_p['location'] = 'entrance'

In [21]:
entrance_p

{'fruit': 0.3774345198119543,
 'dairy': 0.2875755540631296,
 'spices': 0.18146406984553393,
 'drinks': 0.15352585627938214,
 'location': 'entrance'}

In [22]:
# add entrance_p to table: 
trans_prob_2 = trans_prob.reset_index().append(entrance_p, ignore_index=True)

In [23]:
trans_prob_2 = trans_prob_2.set_index('location')

In [24]:
trans_prob_2

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,0.999121,0.000216,0.000248,0.000303,0.000111
dairy,0.061823,0.864568,0.003069,0.035356,0.035183
drinks,0.116492,0.107547,0.598705,0.071664,0.105593
fruit,0.104721,0.033201,0.031932,0.808821,0.021325
spices,0.100893,0.068914,0.063509,0.048495,0.718189
entrance,NaN,0.287576,0.153526,0.377435,0.181464


In [25]:
trans_prob_2.iloc[0].sum(), trans_prob_2.iloc[1].sum(), trans_prob_2.iloc[2].sum(), trans_prob_2.iloc[3].sum(), trans_prob_2.iloc[4].sum(), trans_prob_2.iloc[5].sum()

(1.0, 1.0, 1.0, 0.9999999999999999, 1.0, 1.0)

In [26]:
trans_prob_2.loc['entrance']['checkout'] = 0.0

In [27]:
trans_prob_2

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,0.999121,0.000216,0.000248,0.000303,0.000111
dairy,0.061823,0.864568,0.003069,0.035356,0.035183
drinks,0.116492,0.107547,0.598705,0.071664,0.105593
fruit,0.104721,0.033201,0.031932,0.808821,0.021325
spices,0.100893,0.068914,0.063509,0.048495,0.718189
entrance,0.000000,0.287576,0.153526,0.377435,0.181464


In [28]:
trans_prob_2['entrance'] = [0.00, 0.00, 0.00, 0.00, 0.00, 0.00]

In [29]:
trans_prob_2

,checkout,dairy,drinks,fruit,spices,entrance
location,,,,,,
checkout,0.999121,0.000216,0.000248,0.000303,0.000111,0.0
dairy,0.061823,0.864568,0.003069,0.035356,0.035183,0.0
drinks,0.116492,0.107547,0.598705,0.071664,0.105593,0.0
fruit,0.104721,0.033201,0.031932,0.808821,0.021325,0.0
spices,0.100893,0.068914,0.063509,0.048495,0.718189,0.0
entrance,0.000000,0.287576,0.153526,0.377435,0.181464,0.0


In [30]:
trans_prob_2.to_csv('trans_prob_entrance.csv', index=True)